In [1]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.append('../src')

In [7]:
from get_data import load_history, load_current, combine_history, process_y, load_bios, load_team_data
# load_history()
# load_current()
# combine_history()
# load_team_data()
# load_bios()
# process_y()

import pickle
from model_training import get_pipelines, load_feature_maps
from yahoo_utils import get_games_by_week
from process_data import get_weekly_stats, PRED_COLS

import pandas as pd

current_schedule = get_games_by_week(427)
pipelines = get_pipelines()

preds = {}
skaters_all_feats, skaters_p_feats = load_feature_maps('skater')
sk_p_cols = list(set([c for s in skaters_p_feats.values() for c in s[0]]))
X, y = get_weekly_stats('skater', 
                        cols=sk_p_cols,
                        schedule=current_schedule)
for col in PRED_COLS['skater']:    
    preds[col] = pipelines['skater'][col].predict(X[skaters_all_feats[col][0]])
preds_df = pd.DataFrame(preds, index=X.index)


goalies_all_feats, goalies_p_feats = load_feature_maps('goalie')
gl_p_cols = list(set([c for s in goalies_p_feats.values() for c in s[0]]))

goalie_preds = {}
X, y = get_weekly_stats('goalie', 
                        cols=gl_p_cols,
                        schedule=current_schedule)
for col in PRED_COLS['goalie']:
    goalie_preds[col] = pipelines['goalie'][col].predict(X[goalies_all_feats[col][0]])
    
goalie_preds_df = pd.DataFrame(goalie_preds, index=X.index)


games_df = pd.DataFrame([dict(week=week, **game) for week, games in current_schedule.items() for game in games])

preds_df = preds_df.merge(games_df[['gameId','week']], on='gameId', how='left')\
    .set_index(preds_df.index)
goalie_preds_df = goalie_preds_df.merge(games_df[['gameId','week']], on='gameId', how='left')\
    .set_index(goalie_preds_df.index)

preds_df = pd.concat([preds_df, goalie_preds_df], axis=0)
weekly_preds = preds_df[preds_df['week'] > 0].set_index('week', append=True)
weekly_preds = weekly_preds.drop('gameId', axis=1)

weekly_preds.to_csv('data/2324_preds.csv')

File not found. Fitting...
Fitting for g
Loaded data
0 / 1050
100 / 1050
200 / 1050
300 / 1050
400 / 1050
500 / 1050
600 / 1050
700 / 1050
800 / 1050
900 / 1050
1000 / 1050
Selected feature all_I_F_xGoals_with_earned_rebounds_scoreAdjusted with score 0.07108084245221717
0 / 40
Selected feature all_I_F_goals with score 0.07277820457405626
0 / 32
Selected feature pim with score 0.07405576112545968
0 / 31
Selected feature all_I_F_lowDangerxGoals with score 0.07459362024847149
0 / 24
Selected feature all_I_F_mediumDangerShots with score 0.07566040150506714
0 / 20
Selected feature 5on5_I_F_xPlayContinuedInZone with score 0.07612421800402969
0 / 19
Selected feature 5on5_I_F_takeaways with score 0.07634824790417445
0 / 18
Selected feature position_D with score 0.07646653423478877
0 / 17
Selected feature 5on4_OnIce_F_goals with score 0.07662706777178356
0 / 13
Selected feature 5on4_I_F_playStopped with score 0.07672338097083324
0 / 10
Selected feature other_I_F_oZoneShiftEnds with score 0.0767